In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

import numpy as np
from matplotlib import pyplot as plt
import math
import statistics as stat

import sqlalchemy
from sqlalchemy import create_engine
import psycopg2
from config import db_password

import datetime
from datetime import date
from datetime import time

import re

## Read Kaggle data from the cleaned mls_data_final.csv file

Import the clean `mls_data_final.csv` (originally from Kaggle) data on major league soccer (MLS).

In [3]:
# Read .csv from FinalProject/Resources
file_dir = "mls_data_final.csv"    

# Create a pandas DataFrame 
game_df = pd.read_csv(file_dir, low_memory=False, skipinitialspace = True)

# Show all columns
pd.set_option("display.max_columns", None)

game_df

,id,home,away,date,day,year,venue,home_score,away_score,Outcome,home_goal_scorers,away_goal_scorers,home_possessionPct,away_possessionPct,average_away_possession_from_previous_year,home_total_shots,home_shots_on_goal,away_total_shots,average_away_total_shots_previous_year,away_shots_on_goal,average_away_shots_on_goal_previous_year,home_foulsCommitted,away_foulsCommitted,home_yellowCards,away_yellowCards,home_redCards,away_redCards,average_away_redcards_previous_year,home_offsides,away_offsides,home_wonCorners,away_wonCorners,average_away_corners_won_previous_year,home_saves,away_saves
0,237750,Columbus Crew,Toronto FC,"Saturday, March 29",Saturday,2008,MAPFRE Stadium,2,0,Win,Adam Moffat:Alejandro Moreno,NaN,50%,50%,0.00,9.0,6.0,11.0,NaN,4.0,NaN,13.0,8.0,1.0,1.0,0.0,0.0,NaN,2.0,1.0,3.0,7.0,NaN,8.0,5.0
1,237751,Real Salt Lake,Chicago Fire FC,"Saturday, March 29",Saturday,2008,Rice-Eccles Stadium,1,1,Tie,Bakary Soumare,Cuauhtemoc Blanco,51%,49%,0.00,14.0,7.0,6.0,NaN,1.0,NaN,11.0,5.0,3.0,1.0,0.0,0.0,NaN,1.0,1.0,3.0,3.0,NaN,2.0,6.0
2,237753,Sporting Kansas City,DC United,"Saturday, March 29",Saturday,2008,Community America Ballpark,2,0,Win,Ivan Trujillo:Claudio Lopez,NaN,50%,50%,0.00,7.0,3.0,4.0,NaN,3.0,NaN,4.0,3.0,2.0,2.0,0.0,0.0,NaN,1.0,2.0,2.0,6.0,NaN,3.0,1.0
3,238379,New England Revolution,Houston Dynamo FC,"Saturday, March 29",Saturday,2008,Gillette Stadium,3,0,Win,Steve Ralston:Adam Cristman:Sainey Nyassi,NaN,48%,52%,0.00,12.0,8.0,15.0,NaN,3.0,NaN,6.0,15.0,0.0,1.0,0.0,0.0,NaN,2.0,1.0,4.0,5.0,NaN,5.0,8.0
4,237754,Colorado Rapids,LA Galaxy,"Saturday, March 29",Saturday,2008,Dick's Sporting Goods Park,4,0,Win,Terry Cooke:Christian Gomez:Omar Cummings:Coli...,NaN,52%,48%,0.00,11.0,4.0,5.0,NaN,5.0,NaN,9.0,9.0,0.0,2.0,1.0,1.0,NaN,3.0,4.0,5.0,3.0,NaN,9.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4508,597896,Sporting Kansas City,FC Dallas,"Saturday, July 31",Saturday,2021,Children's Mercy Park,1,2,Loss,Johnny Russell,Paxton Pomykal:Jesús Ferreira,58%,42%,0.49,20.0,4.0,12.0,NaN,5.0,NaN,12.0,14.0,1.0,4.0,0.0,0.0,NaN,2.0,2.0,7.0,2.0,NaN,3.0,3.0
4509,597898,Austin FC,Colorado Rapids,"Saturday, July 31",Saturday,2021,Q2 Stadium,0,1,Loss,NaN,Andre Shinyashiki,57%,43%,0.51,11.0,2.0,16.0,NaN,6.0,NaN,9.0,6.0,2.0,0.0,0.0,0.0,NaN,3.0,3.0,6.0,5.0,NaN,5.0,2.0
4510,597899,Vancouver Whitecaps,Minnesota United FC,"Saturday, July 31",Saturday,2021,Rio Tinto Stadium,2,2,Tie,Cristián Dájome:Cristián Dájome,Ethan Finlay:Robin Lod,52%,48%,0.43,11.0,3.0,13.0,NaN,4.0,NaN,17.0,14.0,3.0,3.0,0.0,0.0,NaN,2.0,3.0,5.0,3.0,NaN,2.0,1.0
4511,597901,Philadelphia Union,Chicago Fire FC,"Sunday, August 1",Sunday,2021,Subaru Park,1,1,Tie,Kai Wagner,Przemyslaw Frankowski,56%,44%,0.52,25.0,5.0,4.0,NaN,2.0,NaN,14.0,8.0,2.0,2.0,0.0,1.0,NaN,2.0,1.0,13.0,1.0,NaN,1.0,4.0


In [4]:
# Get all the column names
game_df.columns

Index(['id', 'home', 'away', 'date', 'day', 'year', 'venue', 'home_score',
       'away_score', 'Outcome', 'home_goal_scorers', 'away_goal_scorers',
       'home_possessionPct', 'away_possessionPct',
       'average_away_possession_from_previous_year', 'home_total_shots',
       'home_shots_on_goal', 'away_total_shots',
       'average_away_total_shots_previous_year', 'away_shots_on_goal',
       'average_away_shots_on_goal_previous_year', 'home_foulsCommitted',
       'away_foulsCommitted', 'home_yellowCards', 'away_yellowCards',
       'home_redCards', 'away_redCards', 'average_away_redcards_previous_year',
       'home_offsides', 'away_offsides', 'home_wonCorners', 'away_wonCorners',
       'average_away_corners_won_previous_year', 'home_saves', 'away_saves'],
      dtype='object')

In [5]:
# Replace the % symbols from these columns
game_df['home_possessionPct'] = game_df['home_possessionPct'].str.rstrip('%').astype('float') / 100
game_df['away_possessionPct'] = game_df['away_possessionPct'].str.rstrip('%').astype('float') / 100
game_df.head(2)

,id,home,away,date,day,year,venue,home_score,away_score,Outcome,home_goal_scorers,away_goal_scorers,home_possessionPct,away_possessionPct,average_away_possession_from_previous_year,home_total_shots,home_shots_on_goal,away_total_shots,average_away_total_shots_previous_year,away_shots_on_goal,average_away_shots_on_goal_previous_year,home_foulsCommitted,away_foulsCommitted,home_yellowCards,away_yellowCards,home_redCards,away_redCards,average_away_redcards_previous_year,home_offsides,away_offsides,home_wonCorners,away_wonCorners,average_away_corners_won_previous_year,home_saves,away_saves
0,237750,Columbus Crew,Toronto FC,"Saturday, March 29",Saturday,2008,MAPFRE Stadium,2,0,Win,Adam Moffat:Alejandro Moreno,NaN,0.50,0.50,0.0,9.0,6.0,11.0,NaN,4.0,NaN,13.0,8.0,1.0,1.0,0.0,0.0,NaN,2.0,1.0,3.0,7.0,NaN,8.0,5.0
1,237751,Real Salt Lake,Chicago Fire FC,"Saturday, March 29",Saturday,2008,Rice-Eccles Stadium,1,1,Tie,Bakary Soumare,Cuauhtemoc Blanco,0.51,0.49,0.0,14.0,7.0,6.0,NaN,1.0,NaN,11.0,5.0,3.0,1.0,0.0,0.0,NaN,1.0,1.0,3.0,3.0,NaN,2.0,6.0


In [6]:
away_team = list(game_df["away"].unique())
away_team

['Toronto FC',
 'Chicago Fire FC',
 'DC United',
 'Houston Dynamo FC',
 'LA Galaxy',
 'Chivas USA',
 'New England Revolution',
 'San Jose Earthquakes',
 'Colorado Rapids',
 'Columbus Crew',
 'Real Salt Lake',
 'FC Dallas',
 'New York Red Bulls',
 'Sporting Kansas City',
 'Seattle Sounders FC',
 'Philadelphia Union',
 'Portland Timbers',
 'Vancouver Whitecaps',
 'CF Montréal',
 'New York City FC',
 'Orlando City SC',
 'Minnesota United FC',
 'Atlanta United FC',
 'LAFC',
 'FC Cincinnati',
 'Inter Miami CF',
 'Nashville SC',
 'Austin FC']

## Group By `away` and `year` column and get the `mean()`

In [7]:
# Group by 'home' and 'year' and calculate average away possession
avg_away_values = game_df.groupby(['away', 'year']).mean()
avg_away_values

id  home_score  away_score  \
away                year                                          
Atlanta United FC   2017  471499.294118    1.235294    1.294118   
                    2018  505254.578947    1.368421    1.526316   
                    2019  533140.529412    1.705882    1.294118   
                    2020  580377.700000    1.800000    1.300000   
                    2021  597681.666667    1.555556    0.888889   
...                                 ...         ...         ...   
Vancouver Whitecaps 2017  473024.666667    1.722222    1.111111   
                    2018  502568.117647    2.235294    1.294118   
                    2019  533132.411765    2.117647    1.235294   
                    2020  578649.615385    1.846154    0.461538   
                    2021  597656.500000    2.125000    1.250000   

                          home_possessionPct  away_possessionPct  \
away                year                                           
Atlanta United FC   2017            0.470000            0.530000   
                    2018            0.500526            0.499474   
                    2019            0.491176            0.508824   
                    2020            0.457000            0.543000   
                    2021            0.427778            0.572222   
...                                      ...                 ...   
Vancouver Whitecaps 2017            0.591111            0.408889   
                    2018            0.610000            0.390000   
                    2019            0.563529            0.436471   
                    2020            0.599231            0.400769   
                    2021            0.556250            0.443750   

                          average_away_possession_from_previous_year  \
away                year                                               
Atlanta United FC   2017                                        0.00   
                    2018                                        0.53   
                    2019                                        0.50   
                    2020                                        0.51   
                    2021                                        0.54   
...                                                              ...   
Vancouver Whitecaps 2017                                        0.43   
                    2018                                        0.41   
                    2019                                        0.39   
                    2020                                        0.44   
                    2021                                        0.40   

                          home_total_shots  home_shots_on_goal  \
away                year                                         
Atlanta United FC   2017         14.823529            4.823529   
                    2018         12.315789            4.842105   
                    2019         13.647059            4.882353   
                    2020         12.500000            4.500000   
                    2021         14.111111            5.777778   
...                                    ...                 ...   
Vancouver Whitecaps 2017         17.000000            5.333333   
                    2018         17.647059            5.411765   
                    2019         22.411765            7.352941   
                    2020         19.615385            6.230769   
                    2021         15.625000            5.375000   

                          away_total_shots  \
away                year                     
Atlanta United FC   2017          9.823529   
                    2018         12.263158   
                    2019         13.823529   
                    2020          9.600000   
                    2021         10.777778   
...                                    ...   
Vancouver Whitecaps 2017         10.055556   
                    2018          9.470588   
                    2019

In [8]:
# As "avg_away_values" are created grouping by 'away' and 'year'
# these 2 columns are not in 'columns' list
avg_away_values.columns

Index(['id', 'home_score', 'away_score', 'home_possessionPct',
       'away_possessionPct', 'average_away_possession_from_previous_year',
       'home_total_shots', 'home_shots_on_goal', 'away_total_shots',
       'average_away_total_shots_previous_year', 'away_shots_on_goal',
       'average_away_shots_on_goal_previous_year', 'home_foulsCommitted',
       'away_foulsCommitted', 'home_yellowCards', 'away_yellowCards',
       'home_redCards', 'away_redCards', 'average_away_redcards_previous_year',
       'home_offsides', 'away_offsides', 'home_wonCorners', 'away_wonCorners',
       'average_away_corners_won_previous_year', 'home_saves', 'away_saves'],
      dtype='object')

In [9]:
# Let's find those 'away', 'year' columns in 'index'
avg_away_values.index

MultiIndex([(  'Atlanta United FC', 2017),
            (  'Atlanta United FC', 2018),
            (  'Atlanta United FC', 2019),
            (  'Atlanta United FC', 2020),
            (  'Atlanta United FC', 2021),
            (          'Austin FC', 2021),
            (        'CF Montréal', 2012),
            (        'CF Montréal', 2013),
            (        'CF Montréal', 2014),
            (        'CF Montréal', 2015),
            ...
            ('Vancouver Whitecaps', 2012),
            ('Vancouver Whitecaps', 2013),
            ('Vancouver Whitecaps', 2014),
            ('Vancouver Whitecaps', 2015),
            ('Vancouver Whitecaps', 2016),
            ('Vancouver Whitecaps', 2017),
            ('Vancouver Whitecaps', 2018),
            ('Vancouver Whitecaps', 2019),
            ('Vancouver Whitecaps', 2020),
            ('Vancouver Whitecaps', 2021)],
           names=['away', 'year'], length=282)

In [10]:
# Convert this index to a list to get the values easily
avg_away_list = list(avg_away_values.index)
avg_away_list

[('Atlanta United FC', 2017),
 ('Atlanta United FC', 2018),
 ('Atlanta United FC', 2019),
 ('Atlanta United FC', 2020),
 ('Atlanta United FC', 2021),
 ('Austin FC', 2021),
 ('CF Montréal', 2012),
 ('CF Montréal', 2013),
 ('CF Montréal', 2014),
 ('CF Montréal', 2015),
 ('CF Montréal', 2016),
 ('CF Montréal', 2017),
 ('CF Montréal', 2018),
 ('CF Montréal', 2019),
 ('CF Montréal', 2020),
 ('CF Montréal', 2021),
 ('Chicago Fire FC', 2008),
 ('Chicago Fire FC', 2009),
 ('Chicago Fire FC', 2010),
 ('Chicago Fire FC', 2011),
 ('Chicago Fire FC', 2012),
 ('Chicago Fire FC', 2013),
 ('Chicago Fire FC', 2014),
 ('Chicago Fire FC', 2015),
 ('Chicago Fire FC', 2016),
 ('Chicago Fire FC', 2017),
 ('Chicago Fire FC', 2018),
 ('Chicago Fire FC', 2019),
 ('Chicago Fire FC', 2020),
 ('Chicago Fire FC', 2021),
 ('Chivas USA', 2008),
 ('Chivas USA', 2009),
 ('Chivas USA', 2010),
 ('Chivas USA', 2011),
 ('Chivas USA', 2012),
 ('Chivas USA', 2013),
 ('Chivas USA', 2014),
 ('Colorado Rapids', 2008),
 ('Colo

In [11]:
# For example: To get the "team name"
avg_away_list[5][0]

'Austin FC'

In [17]:
# For example: to get the "year"
avg_away_list[0][1]

2017

In [217]:
#for i in range(5):
    #df = df.append({'A': i}, ignore_index=True)
len(avg_away_list)

282

In [106]:
# Fill in the "average_away_total_shots_previous_year" column with data from 
        # the "away_total_shots" column in the "grouped table" grouped by "away" & "year"

for j in range(len(game_df['away'])) : # len(game_df['away']) = 4513 ; no. of Rows in original 'game_df'
    for i in range(282) : # len(avg_away_list) = 282 ; no. of Rows in grouped by 'away', 'year' Table
        game_df['average_away_total_shots_previous_year'][j] = np.where(((game_df.away[j] == avg_away_list[i][0]) \
                            and game_df.year[j] == avg_away_list[i][1]), avg_away_values.away_total_shots[i], \
                                                             game_df.average_away_total_shots_previous_year[j])
game_df

C:\Users\mamoo\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,id,home,away,date,day,year,venue,home_score,away_score,Outcome,home_goal_scorers,away_goal_scorers,home_possessionPct,away_possessionPct,average_away_possession_from_previous_year,home_total_shots,home_shots_on_goal,away_total_shots,average_away_total_shots_previous_year,away_shots_on_goal,average_away_shots_on_goal_previous_year,home_foulsCommitted,away_foulsCommitted,home_yellowCards,away_yellowCards,home_redCards,away_redCards,average_away_redcards_previous_year,home_offsides,away_offsides,home_wonCorners,away_wonCorners,average_away_corners_won_previous_year,home_saves,away_saves
0,237750,Columbus Crew,Toronto FC,"Saturday, March 29",Saturday,2008,MAPFRE Stadium,2,0,Win,Adam Moffat:Alejandro Moreno,NaN,0.50,0.50,0.00,13.0,6.0,13.0,7.133333,4.0,NaN,13.0,13.0,1.0,1.0,0.0,0.0,NaN,2.0,1.0,3.0,7.0,NaN,13.0,5.0
1,237751,Real Salt Lake,Chicago Fire FC,"Saturday, March 29",Saturday,2008,Rice-Eccles Stadium,1,1,Tie,Bakary Soumare,Cuauhtemoc Blanco,0.51,0.49,0.00,14.0,7.0,6.0,8.235294,1.0,NaN,13.0,5.0,3.0,1.0,0.0,0.0,NaN,1.0,1.0,3.0,3.0,NaN,2.0,6.0
2,237753,Sporting Kansas City,DC United,"Saturday, March 29",Saturday,2008,Community America Ballpark,2,0,Win,Ivan Trujillo:Claudio Lopez,NaN,0.50,0.50,0.00,7.0,3.0,4.0,7.400000,3.0,NaN,4.0,3.0,2.0,2.0,0.0,0.0,NaN,1.0,2.0,2.0,6.0,NaN,3.0,1.0
3,238379,New England Revolution,Houston Dynamo FC,"Saturday, March 29",Saturday,2008,Gillette Stadium,3,0,Win,Steve Ralston:Adam Cristman:Sainey Nyassi,NaN,0.48,0.52,0.00,13.0,13.0,15.0,10.352941,3.0,NaN,6.0,15.0,0.0,1.0,0.0,0.0,NaN,2.0,1.0,4.0,5.0,NaN,5.0,13.0
4,237754,Colorado Rapids,LA Galaxy,"Saturday, March 29",Saturday,2008,Dick's Sporting Goods Park,4,0,Win,Terry Cooke:Christian Gomez:Omar Cummings:Coli...,NaN,0.52,0.48,0.00,13.0,4.0,5.0,10.066667,5.0,NaN,13.0,13.0,0.0,2.0,1.0,1.0,NaN,3.0,4.0,5.0,3.0,NaN,13.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4508,597896,Sporting Kansas City,FC Dallas,"Saturday, July 31",Saturday,2021,Children's Mercy Park,1,2,Loss,Johnny Russell,Paxton Pomykal:Jesús Ferreira,0.58,0.42,0.49,20.0,4.0,13.0,11.250000,5.0,NaN,13.0,14.0,1.0,4.0,0.0,0.0,NaN,2.0,2.0,7.0,2.0,NaN,3.0,3.0
4509,597898,Austin FC,Colorado Rapids,"Saturday, July 31",Saturday,2021,Q2 Stadium,0,1,Loss,NaN,Andre Shinyashiki,0.57,0.43,0.51,13.0,2.0,16.0,11.142857,6.0,NaN,13.0,6.0,2.0,0.0,0.0,0.0,NaN,3.0,3.0,6.0,5.0,NaN,5.0,2.0
4510,597899,Vancouver Whitecaps,Minnesota United FC,"Saturday, July 31",Saturday,2021,Rio Tinto Stadium,2,2,Tie,Cristián Dájome:Cristián Dájome,Ethan Finlay:Robin Lod,0.52,0.48,0.43,13.0,3.0,13.0,11.750000,4.0,NaN,17.0,14.0,3.0,3.0,0.0,0.0,NaN,2.0,3.0,5.0,3.0,NaN,2.0,1.0
4511,597901,Philadelphia Union,Chicago Fire FC,"Sunday, August 1",Sunday,2021,Subaru Park,1,1,Tie,Kai Wagner,Przemyslaw Frankowski,0.56,0.44,0.52,25.0,5.0,4.0,11.000000,2.0,NaN,14.0,13.0,2.0,2.0,0.0,1.0,NaN,2.0,1.0,13.0,1.0,NaN,1.0,4.0


In [107]:
for j in range(len(game_df['away'])) : # len(game_df['away']) = 4513 ; no. of Rows in original 'game_df'
    for i in range(len(avg_away_list)) : # len(avg_away_list) = 282 ; no. of Rows in grouped by 'away', 'year' Table
        
        # Fill in the "average_away_possession_from_previous_year" column with data from 
        # the "away_possessionPct" column in the "grouped table" grouped by "away" & "year"
        game_df['average_away_possession_from_previous_year'][j] = np.where(((game_df.away[j] == avg_away_list[i][0]) \
                            and game_df.year[j] == avg_away_list[i][1]), avg_away_values.away_possessionPct[i], \
                                                             game_df.average_away_possession_from_previous_year[j])
        
        # Fill in the "average_away_shots_on_goal_previous_year" column with data from 
        # the "away_shots_on_goal" column in the "grouped table" grouped by "away" & "year"
        game_df['average_away_shots_on_goal_previous_year'][j] = np.where(((game_df.away[j] == avg_away_list[i][0]) \
                            and game_df.year[j] == avg_away_list[i][1]), avg_away_values.away_shots_on_goal[i], \
                                                             game_df.average_away_shots_on_goal_previous_year[j])
        
        # Fill in the "average_away_redcards_pervious_year" column with data from 
        # the "away_redCards" column in the "grouped table" grouped by "away" & "year"
        game_df['average_away_redcards_previous_year'][j] = np.where(((game_df.away[j] == avg_away_list[i][0]) \
                            and game_df.year[j] == avg_away_list[i][1]), avg_away_values.away_redCards[i], \
                                                             game_df.average_away_redcards_previous_year[j])
        
        # Fill in the "average_away_corners_won_previous_year" column with data from 
        # the "away_wonCorners" column in the "grouped table" grouped by "away" & "year"
        game_df['average_away_corners_won_previous_year'][j] = np.where(((game_df.away[j] == avg_away_list[i][0]) \
                            and game_df.year[j] == avg_away_list[i][1]), avg_away_values.away_wonCorners[i], \
                                                             game_df.average_away_corners_won_previous_year[j])
        
        
game_df

C:\Users\mamoo\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\mamoo\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\mamoo\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\mamoo\anaconda3\envs\PythonData\lib\si

,id,home,away,date,day,year,venue,home_score,away_score,Outcome,home_goal_scorers,away_goal_scorers,home_possessionPct,away_possessionPct,average_away_possession_from_previous_year,home_total_shots,home_shots_on_goal,away_total_shots,average_away_total_shots_previous_year,away_shots_on_goal,average_away_shots_on_goal_previous_year,home_foulsCommitted,away_foulsCommitted,home_yellowCards,away_yellowCards,home_redCards,away_redCards,average_away_redcards_previous_year,home_offsides,away_offsides,home_wonCorners,away_wonCorners,average_away_corners_won_previous_year,home_saves,away_saves
0,237750,Columbus Crew,Toronto FC,"Saturday, March 29",Saturday,2008,MAPFRE Stadium,2,0,Win,Adam Moffat:Alejandro Moreno,NaN,0.50,0.50,0.498000,13.0,6.0,13.0,7.133333,4.0,3.400000,13.0,13.0,1.0,1.0,0.0,0.0,0.200000,2.0,1.0,3.0,7.0,3.866667,13.0,5.0
1,237751,Real Salt Lake,Chicago Fire FC,"Saturday, March 29",Saturday,2008,Rice-Eccles Stadium,1,1,Tie,Bakary Soumare,Cuauhtemoc Blanco,0.51,0.49,0.488235,14.0,7.0,6.0,8.235294,1.0,4.176471,13.0,5.0,3.0,1.0,0.0,0.0,0.000000,1.0,1.0,3.0,3.0,3.411765,2.0,6.0
2,237753,Sporting Kansas City,DC United,"Saturday, March 29",Saturday,2008,Community America Ballpark,2,0,Win,Ivan Trujillo:Claudio Lopez,NaN,0.50,0.50,0.507333,7.0,3.0,4.0,7.400000,3.0,3.733333,4.0,3.0,2.0,2.0,0.0,0.0,0.266667,1.0,2.0,2.0,6.0,4.333333,3.0,1.0
3,238379,New England Revolution,Houston Dynamo FC,"Saturday, March 29",Saturday,2008,Gillette Stadium,3,0,Win,Steve Ralston:Adam Cristman:Sainey Nyassi,NaN,0.48,0.52,0.517059,13.0,13.0,15.0,10.352941,3.0,4.235294,6.0,15.0,0.0,1.0,0.0,0.0,0.117647,2.0,1.0,4.0,5.0,4.411765,5.0,13.0
4,237754,Colorado Rapids,LA Galaxy,"Saturday, March 29",Saturday,2008,Dick's Sporting Goods Park,4,0,Win,Terry Cooke:Christian Gomez:Omar Cummings:Coli...,NaN,0.52,0.48,0.492667,13.0,4.0,5.0,10.066667,5.0,5.133333,13.0,13.0,0.0,2.0,1.0,1.0,0.066667,3.0,4.0,5.0,3.0,3.600000,13.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4508,597896,Sporting Kansas City,FC Dallas,"Saturday, July 31",Saturday,2021,Children's Mercy Park,1,2,Loss,Johnny Russell,Paxton Pomykal:Jesús Ferreira,0.58,0.42,0.515000,20.0,4.0,13.0,11.250000,5.0,3.375000,13.0,14.0,1.0,4.0,0.0,0.0,0.000000,2.0,2.0,7.0,2.0,3.375000,3.0,3.0
4509,597898,Austin FC,Colorado Rapids,"Saturday, July 31",Saturday,2021,Q2 Stadium,0,1,Loss,NaN,Andre Shinyashiki,0.57,0.43,0.460000,13.0,2.0,16.0,11.142857,6.0,3.857143,13.0,6.0,2.0,0.0,0.0,0.0,0.000000,3.0,3.0,6.0,5.0,4.285714,5.0,2.0
4510,597899,Vancouver Whitecaps,Minnesota United FC,"Saturday, July 31",Saturday,2021,Rio Tinto Stadium,2,2,Tie,Cristián Dájome:Cristián Dájome,Ethan Finlay:Robin Lod,0.52,0.48,0.478750,13.0,3.0,13.0,11.750000,4.0,4.125000,17.0,14.0,3.0,3.0,0.0,0.0,0.125000,2.0,3.0,5.0,3.0,3.625000,2.0,1.0
4511,597901,Philadelphia Union,Chicago Fire FC,"Sunday, August 1",Sunday,2021,Subaru Park,1,1,Tie,Kai Wagner,Przemyslaw Frankowski,0.56,0.44,0.503333,25.0,5.0,4.0,11.000000,2.0,3.000000,14.0,13.0,2.0,2.0,0.0,1.0,0.333333,2.0,1.0,13.0,1.0,3.000000,1.0,4.0


In [108]:
game_df.columns

Index(['id', 'home', 'away', 'date', 'day', 'year', 'venue', 'home_score',
       'away_score', 'Outcome', 'home_goal_scorers', 'away_goal_scorers',
       'home_possessionPct', 'away_possessionPct',
       'average_away_possession_from_previous_year', 'home_total_shots',
       'home_shots_on_goal', 'away_total_shots',
       'average_away_total_shots_previous_year', 'away_shots_on_goal',
       'average_away_shots_on_goal_previous_year', 'home_foulsCommitted',
       'away_foulsCommitted', 'home_yellowCards', 'away_yellowCards',
       'home_redCards', 'away_redCards', 'average_away_redcards_previous_year',
       'home_offsides', 'away_offsides', 'home_wonCorners', 'away_wonCorners',
       'average_away_corners_won_previous_year', 'home_saves', 'away_saves'],
      dtype='object')

In [109]:
# Drop columns with data those are not easily known before a game starts.
game_df = game_df.drop(['id', 'date', 'home_goal_scorers', 'away_goal_scorers',
       'home_possessionPct', 'away_possessionPct',
       'home_total_shots', 'home_shots_on_goal', 'away_total_shots',
       'away_shots_on_goal', 'home_foulsCommitted',
       'away_foulsCommitted', 'home_yellowCards', 'away_yellowCards',
       'home_redCards', 'away_redCards', 'home_offsides', 'away_offsides', 'home_wonCorners', 'away_wonCorners',
       'home_saves', 'away_saves'], axis='columns')
game_df.head(2)

,home,away,day,year,venue,home_score,away_score,Outcome,average_away_possession_from_previous_year,average_away_total_shots_previous_year,average_away_shots_on_goal_previous_year,average_away_redcards_previous_year,average_away_corners_won_previous_year
0,Columbus Crew,Toronto FC,Saturday,2008,MAPFRE Stadium,2,0,Win,0.498000,7.133333,3.400000,0.2,3.866667
1,Real Salt Lake,Chicago Fire FC,Saturday,2008,Rice-Eccles Stadium,1,1,Tie,0.488235,8.235294,4.176471,0.0,3.411765


In [111]:
# Export this table (with grouped data for calcualted "average away - shots, red cards etc.") to .csv
file_path = "../Resources/Clean_csv/"
game_df.to_csv(f"{file_path}mls_away_avg.csv")

In [57]:
game_df.columns

Index(['home', 'away', 'day', 'year', 'venue', 'home_score', 'away_score',
       'Outcome', 'average_away_possession_from_previous_year'],
      dtype='object')

In [58]:
# Find the total null values in each column
game_df.isnull().sum()

home                                            0
away                                            0
day                                             0
year                                            0
venue                                           1
home_score                                      0
away_score                                      0
Outcome                                         0
average_away_possession_from_previous_year    115
dtype: int64

In [60]:
# Drop null entries
game_df = game_df.dropna()

In [61]:
game_df.isnull().sum()

home                                          0
away                                          0
day                                           0
year                                          0
venue                                         0
home_score                                    0
away_score                                    0
Outcome                                       0
average_away_possession_from_previous_year    0
dtype: int64

In [62]:
# No. of unique data in each column
game_df.nunique()

home                                          28
away                                          27
day                                            7
year                                          14
venue                                         59
home_score                                     8
away_score                                     8
Outcome                                        3
average_away_possession_from_previous_year    21
dtype: int64

In [63]:
game_df.dtypes

home                                           object
away                                           object
day                                            object
year                                            int64
venue                                          object
home_score                                      int64
away_score                                      int64
Outcome                                        object
average_away_possession_from_previous_year    float64
dtype: object

It looks like there are 5 columns with **categorical** values having consistent data type as *object*.

We should **generate a list of categorical variable** names using Python's "df.dtypes" property. In that case, we can use our variable list to perform the **one-hot encoding** *once*, rather than for each individual variable.

In [64]:
# Generate our categorical variable list
game_cat = game_df.dtypes[game_df.dtypes == "object"].index.tolist()
game_cat

['home', 'away', 'day', 'venue', 'Outcome']

In [65]:
# Check the number of unique values in each column
game_df[game_cat].nunique()

home       28
away       27
day         7
venue      59
Outcome     3
dtype: int64

In [66]:
# Check the home team names
game_df['home'].unique()

array(['Columbus Crew', 'Real Salt Lake', 'Sporting Kansas City',
       'New England Revolution', 'Colorado Rapids', 'Chicago Fire FC',
       'LA Galaxy', 'DC United', 'New York Red Bulls', 'Chivas USA',
       'Houston Dynamo FC', 'San Jose Earthquakes', 'FC Dallas',
       'Toronto FC', 'Seattle Sounders FC', 'Philadelphia Union',
       'Vancouver Whitecaps', 'Portland Timbers', 'CF Montréal',
       'Orlando City SC', 'New York City FC', 'Atlanta United FC',
       'Minnesota United FC', 'LAFC', 'FC Cincinnati', 'Nashville SC',
       'Inter Miami CF', 'Austin FC'], dtype=object)

Before going to encode these columns using Scikit-learn's OneHotEncoder module, we need to make sure that if the categorical variables require **bucketing**. Since we want to predict outcome for each individual `home` team, we need to see the relation of all individual `home` team, `venue` and other columns, we choose not to to any binning/bucketing, and we're ready to use **OneHotEncoder**.

In [67]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(game_df[game_cat]))
encode_df.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [68]:
# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(game_cat)
encode_df.head()

,home_Atlanta United FC,home_Austin FC,home_CF Montréal,home_Chicago Fire FC,home_Chivas USA,home_Colorado Rapids,home_Columbus Crew,home_DC United,home_FC Cincinnati,home_FC Dallas,home_Houston Dynamo FC,home_Inter Miami CF,home_LA Galaxy,home_LAFC,home_Minnesota United FC,home_Nashville SC,home_New England Revolution,home_New York City FC,home_New York Red Bulls,home_Orlando City SC,home_Philadelphia Union,home_Portland Timbers,home_Real Salt Lake,home_San Jose Earthquakes,home_Seattle Sounders FC,home_Sporting Kansas City,home_Toronto FC,home_Vancouver Whitecaps,away_Atlanta United FC,away_Austin FC,away_CF Montréal,away_Chicago Fire FC,away_Colorado Rapids,away_Columbus Crew,away_DC United,away_FC Cincinnati,away_FC Dallas,away_Houston Dynamo FC,away_Inter Miami CF,away_LA Galaxy,away_LAFC,away_Minnesota United FC,away_Nashville SC,away_New England Revolution,away_New York City FC,away_New York Red Bulls,away_Orlando City SC,away_Philadelphia Union,away_Portland Timbers,away_Real Salt Lake,away_San Jose Earthquakes,away_Seattle Sounders FC,away_Sporting Kansas City,away_Toronto FC,away_Vancouver Whitecaps,day_Friday,day_Monday,day_Saturday,day_Sunday,day_Thursday,day_Tuesday,day_Wednesday,venue_AT&T Stadium,venue_Allianz Field,venue_Audi Field,venue_BBVA Stadium,venue_BC Place,venue_BMO Field,venue_Banc of California Stadium,venue_Bobby Dodd Stadium,venue_Buck Shaw Stadium,venue_Camping World Stadium,venue_CenturyLink Field,venue_Children's Mercy Park,venue_Citi Field Stadium,venue_Community America Ballpark,venue_DRV PNK Stadium,venue_Dick's Sporting Goods Park,venue_Dignity Health Sports Park,venue_ESPN Wide World of Sports Complex,venue_Earthquakes Stadium,venue_Empire Field,venue_Exploria Stadium,venue_FedExField,venue_Giants Stadium,venue_Gillette Stadium,venue_Historic Crew Stadium,venue_Inter Miami CF Stadium,venue_Levi's Stadium,venue_Lower.com Field,venue_Lumen Field,venue_MAPFRE Stadium,venue_Maryland SoccerPlex,venue_McAfee Coliseum,venue_Mercedes-Benz Stadium,venue_Navy-Marine Corps Memorial Stadium,venue_Nippert Stadium,venue_Nissan Stadium,venue_PayPal Park,venue_Pizza Hut Park,venue_Pratt & Whitney Stadium at Rentschler Field,venue_Providence Park,venue_Q2 Stadium,venue_Qwest Field,venue_RFK Stadium,venue_Red Bull Arena,venue_Rice-Eccles Stadium,venue_Rio Tinto Stadium,venue_Robertson Stadium,venue_Rogers Centere,venue_SeatGeek Stadium,venue_Soldier Field,venue_Stade Olympique,venue_Stade Saputo,venue_Stanford Stadium,venue_StubHub Center,venue_Subaru Park,venue_TCF Bank Stadium,venue_TQL Stadium,venue_Toyota Stadium,venue_Yankee Stadium,Outcome_Loss,Outcome_Tie,Outcome_Win
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

Now that our categorical variables have been encoded, they are ready to replace our unencoded categorical variables in our dataset using *panda's* **merge** and **drop** methods.

In [69]:
# Merge OneHotEncoded features and drop the originals from the updated df
game_df = game_df.merge(encode_df, left_index=True, right_index=True).drop(game_df[game_cat],1)
game_df.head()

,year,home_score,away_score,average_away_possession_from_previous_year,home_Atlanta United FC,home_Austin FC,home_CF Montréal,home_Chicago Fire FC,home_Chivas USA,home_Colorado Rapids,home_Columbus Crew,home_DC United,home_FC Cincinnati,home_FC Dallas,home_Houston Dynamo FC,home_Inter Miami CF,home_LA Galaxy,home_LAFC,home_Minnesota United FC,home_Nashville SC,home_New England Revolution,home_New York City FC,home_New York Red Bulls,home_Orlando City SC,home_Philadelphia Union,home_Portland Timbers,home_Real Salt Lake,home_San Jose Earthquakes,home_Seattle Sounders FC,home_Sporting Kansas City,home_Toronto FC,home_Vancouver Whitecaps,away_Atlanta United FC,away_Austin FC,away_CF Montréal,away_Chicago Fire FC,away_Colorado Rapids,away_Columbus Crew,away_DC United,away_FC Cincinnati,away_FC Dallas,away_Houston Dynamo FC,away_Inter Miami CF,away_LA Galaxy,away_LAFC,away_Minnesota United FC,away_Nashville SC,away_New England Revolution,away_New York City FC,away_New York Red Bulls,away_Orlando City SC,away_Philadelphia Union,away_Portland Timbers,away_Real Salt Lake,away_San Jose Earthquakes,away_Seattle Sounders FC,away_Sporting Kansas City,away_Toronto FC,away_Vancouver Whitecaps,day_Friday,day_Monday,day_Saturday,day_Sunday,day_Thursday,day_Tuesday,day_Wednesday,venue_AT&T Stadium,venue_Allianz Field,venue_Audi Field,venue_BBVA Stadium,venue_BC Place,venue_BMO Field,venue_Banc of California Stadium,venue_Bobby Dodd Stadium,venue_Buck Shaw Stadium,venue_Camping World Stadium,venue_CenturyLink Field,venue_Children's Mercy Park,venue_Citi Field Stadium,venue_Community America Ballpark,venue_DRV PNK Stadium,venue_Dick's Sporting Goods Park,venue_Dignity Health Sports Park,venue_ESPN Wide World of Sports Complex,venue_Earthquakes Stadium,venue_Empire Field,venue_Exploria Stadium,venue_FedExField,venue_Giants Stadium,venue_Gillette Stadium,venue_Historic Crew Stadium,venue_Inter Miami CF Stadium,venue_Levi's Stadium,venue_Lower.com Field,venue_Lumen Field,venue_MAPFRE Stadium,venue_Maryland SoccerPlex,venue_McAfee Coliseum,venue_Mercedes-Benz Stadium,venue_Navy-Marine Corps Memorial Stadium,venue_Nippert Stadium,venue_Nissan Stadium,venue_PayPal Park,venue_Pizza Hut Park,venue_Pratt & Whitney Stadium at Rentschler Field,venue_Providence Park,venue_Q2 Stadium,venue_Qwest Field,venue_RFK Stadium,venue_Red Bull Arena,venue_Rice-Eccles Stadium,venue_Rio Tinto Stadium,venue_Robertson Stadium,venue_Rogers Centere,venue_SeatGeek Stadium,venue_Soldier Field,venue_Stade Olympique,venue_Stade Saputo,venue_Stanford Stadium,venue_StubHub Center,venue_Subaru Park,venue_TCF Bank Stadium,venue_TQL Stadium,venue_Toyota Stadium,venue_Yankee Stadium,Outcome_Loss,Outcome_Tie,Outcome_Win
0,2008,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2008,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2008,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0

We need to **split** our **training** and **testing** data *before* fitting our **StandardScaler** instance. This <u> prevents testing data from influencing the standardization </u> function.

To build our training and testing datasets, we need to separate two values:

input values (which are our *independent variables* commonly referred to as **model features or "X"**) and **target output** ( *dependent variable* commonly referred to as **target or "y"** in TensorFlow documentation).

We want to build a model that will predict whether or not a team is winning; therefore, we must separate the `Outcome_win` column from the rest of the input data. Also, the `Outcome_loss`, `Outcome_tie`columns have the same input for all the data, so we can drop this column.

In [70]:
# Split our preprocessed data into our features and target arrays
y = game_df["Outcome_Win"].values
X = game_df.drop(["Outcome_Win","Outcome_Loss", "Outcome_Tie" ], 1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

Now that our training and testing data have been allocated, we're ready to build our **StandardScalerobject** and standardize the numerical features.

In [71]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

Our data is now **preprocessed** via one-hot encoding and standardization. 

### Define Neural Network model

For our **input layer**, we must add the **number of input features equal to the number of variables in our feature** DataFrame.

In our **hidden layers**, we'll add **three hidden layers** with only a few neurons in each layer. To create the *second hidden layer*, we'll add another **Keras Dense class** while defining our model. All of our hidden layers will use the **relu activation** function to identify nonlinear characteristics from the input values.

In the **output layer**, we'll use the `sigmoid` activation function that will help us predict the probability that a team is winning or not.

In [72]:
len(X_train[0])

125

In [73]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  250
hidden_nodes_layer2 = 125
hidden_nodes_layer3 = 25

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 250)               31500     
_________________________________________________________________
dense_5 (Dense)              (None, 125)               31375     
_________________________________________________________________
dense_6 (Dense)              (None, 25)                3150      
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 26        
Total params: 66,051
Trainable params: 66,051
Non-trainable params: 0
_________________________________________________________________


In [74]:
# Compile the model
#nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
nn.compile(loss="mean_squared_error", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
101/101 [==============================] - 1s 5ms/step - loss: 0.2520 - accuracy: 0.5271
Epoch 2/100
101/101 [==============================] - 0s 4ms/step - loss: 0.2384 - accuracy: 0.5860
Epoch 3/100
101/101 [==============================] - 0s 4ms/step - loss: 0.2309 - accuracy: 0.6159
Epoch 4/100
101/101 [==============================] - 0s 4ms/step - loss: 0.2203 - accuracy: 0.6449
Epoch 5/100
101/101 [==============================] - 0s 4ms/step - loss: 0.2122 - accuracy: 0.6657
Epoch 6/100
101/101 [==============================] - 1s 5ms/step - loss: 0.1976 - accuracy: 0.7037
Epoch 7/100
101/101 [==============================] - 1s 7ms/step - loss: 0.1852 - accuracy: 0.7259
Epoch 8/100
101/101 [==============================] - 1s 7ms/step - loss: 0.1785 - accuracy: 0.7389: 0s - loss: 0.1
Epoch 9/100
101/101 [==============================] - 1s 6ms/step - loss: 0.1635 - accuracy: 0.7623
Epoch 10/100
101/101 [==============================] - 1s 9ms/step - loss:

101/101 [==============================] - 1s 10ms/step - loss: 0.0180 - accuracy: 0.9785 0s - l
Epoch 76/100
101/101 [==============================] - 1s 11ms/step - loss: 0.0142 - accuracy: 0.9829
Epoch 77/100
101/101 [==============================] - ETA: 0s - loss: 0.0140 - accuracy: 0.9828 ETA: 0s - loss: 0.0124 - accuracy: 0. - 1s 9ms/step - loss: 0.0139 - accuracy: 0.9829
Epoch 78/100
101/101 [==============================] - 1s 10ms/step - loss: 0.0130 - accuracy: 0.9832
Epoch 79/100
101/101 [==============================] - 1s 9ms/step - loss: 0.0117 - accuracy: 0.9854
Epoch 80/100
101/101 [==============================] - 1s 12ms/step - loss: 0.0102 - accuracy: 0.9854
Epoch 81/100
101/101 [==============================] - 1s 12ms/step - loss: 0.0133 - accuracy: 0.9832
Epoch 82/100
101/101 [==============================] - 1s 12ms/step - loss: 0.0124 - accuracy: 0.9847
Epoch 83/100
101/101 [==============================] - 1s 12ms/step - loss: 0.0110 - accuracy: 0.9869